http://cl-informatik.uibk.ac.at/isr24/ rewriting school. Seems like all the good topics
https://link.springer.com/book/10.1007/978-3-031-43369-6 frontiers of combining systems

polynomial interpretations
matrix interpretations
SAT/LP solver for termination
dealing wiht the xml format
How does the frontend deal with things

There is rewriting modulo equations, realtive rewriting, innermost rewriting, conditional rewriting, outermost rewriting, higher order rewriting hrs, cycle rewriting, string rewriting. How are these dealt with?

Confluence checking CSI
https://en.wikipedia.org/wiki/Cycle_detection

Java. Hmm.
eBPF

https://maude.cs.uiuc.edu/tools/mta/
https://maude.cs.uiuc.edu/tools/itp/hendrix-thesis.pdf
hendrix thesis. Equational tree automata

Well founded induction

if foo(s(x)) -> foo(x) is a well founded definition, then there is intrinisically a valid induction hypothesis in there
Is extracting the piece related the dependency pairs?
How to deal with completeness check? Default to some dummy value? Well, it isn't a problem. One can't prove what the value is on other pieces. We do want confluence though. To make sure we don't have foo(x) == true /\ foo(x) == false

higher order. sizechangetool  https://github.com/Deducteam/SizeChangeTool wanda 

## Dependency Pairs
https://www.sciencedirect.com/science/article/pii/S0304397599002078 Termination of term rewriting using dependency pairs
Dependency pairs  fraemwork http://cl-informatik.uibk.ac.at/users/thiemann/paper/LPAR04DPFramework.pdf

https://drops.dagstuhl.de/storage/00lipics/lipics-vol131-fscd2019/LIPIcs.FSCD.2019.9/LIPIcs.FSCD.2019.9.pdf Dependency Pairs Termination in Dependent Type Theory Modulo Rewriting

Wanda 

"Dependency pairs aren't that mysterious:
Plus(x, S(y)) -> S(Plus(x, y))
gives the pair
Plus(x, S(y)) -> Plus(x, y)
which intuitively is the only bit we really care about re termination.
This is the "ignore the context" bit you were mentioning.
"

"
Dependency pairs: not that tough an idea
take a rewrite system R = { l -> r }
Define G to be like R, but with every possible subterm of each right hand side
so if G is f(x, y) -> g(f(y, x), y) then G is f(x, y) -> g(f(y, x), y) and f(x, y) -> f(y, x)
now only allow G steps at the root, and then some arbitrary number of R steps not at the root
the theorem is that termination of R <=> there is no infinite reduction of G-root + R non-root steps
the idea is that G "focuses" the reductions on each possible subterm, ignoring the context
now you only need to care about possible infinite sequences of G steps (with in-between R steps)

Seems worse
G is bigger than r

yes but
you can observe that some G steps may not follow another G step
e.g.
times(x, S(y)) -> plus(x, times(x, y))
the G steps are times(x, S(y)) -> plus(x, times(x, y)) and times(x, S(y)) -> times(x, y)
but there is no outer times rule for times after the first step!
so the only "loop" for times is the second one
the idea is that you can treat each "mutual recursion" rules individually
often to make this explicit they rename the G head symbols
so G is actually TIMES(x, S(y)) -> PLUS(x, times(x, y)) and TIMES(x, S(y)) -> TIMES(x, y)
which feels different
there's  only one rules that can lead to infinite TIMES steps

But we still have all of R

but what you need to show is termination of alternating G/R steps
so if you prove that there can't be infinitely many G steps you're golden
you'll see that in AproVE that's what they're trying to do
"remove" G rules
in practice you replace "non-trivial" inner terms by some fresh var 
so TIMES(x, S(y)) -> PLUS(x, z)
that means any plus rule can fire at this point
but a PLUS rule can't reach a TIMES rule, so you can prove first termination of PLUS then termination of TIMES
but read the paper it's well-written

Plus doesnt depend on times. Is that not sufficient to see they can be treated hierarchically?
I cant show plus terminates then leave its rules out and show times terminates?
How could termination checking possibly work in coq then

Yes you can, but in TRSes things are not generally so modular
"depends on" is basically what the dependency pair relation captures
you can have crazy rules like plus(x, x) -> times(2, x)
does that terminate?
anyways

I’m suspicious."

simplification pair 
reduction order
order eakly, delete strict pieces
SCC dependency graph


https://aprove.informatik.rwth-aachen.de/download

https://matryoshka-project.github.io/wait2018/ WAIT Workshop on Automated (Co)inductive Theorem Proving
HipSpec https://www.cse.chalmers.se/~jomoa/papers/hipspec-atx.pdf
https://dmg.tuwien.ac.at/hetzl/teaching/d_vierling.pdf The limits of automated
inductive theorem provers 2023 thesis
vampire has induction support. What does it look like?
https://www.cse.chalmers.se/~jomoa/papers/cicm15-TIP.pdf tons of inductive problems https://github.com/tip-org


A core method of termination is to solve for a term order that meets your needs.
If you're given a pile of equations, you could need to make choices about which direction they should point.

For string rewriting / string equations variables don't matter that much

It may be useful to maxsat this termination rather than insist it simply works.


In [73]:
#import cvxpy
import z3
from dataclasses import dataclass
from collections import namedtuple

Var = namedtuple("Var", "name")
Fn = namedtuple("Fn", "head args")
#(RULES
#    plus(0,y) -> y
#    plus(s(x),y) -> s(plus(x,y))
#)
rules = [
    (Fn("f", [Var("x")]), Var("x")),
    (Fn("plus", [Fn("s", [Var("x")]), Var("y")]), Fn("s", [Fn("plus", [Var("x"), Var("y")])])),
    (Fn("plus", [Fn("0", []), Var("y")]), Var("x")),
]

def weight(t):
    if isinstance(t, Var):
        return z3.Int("__varweight")
    elif isinstance(t, Fn):
        return z3.Int(t.head) + sum(weight(a) for a in t.args)
    else:
        raise ValueError(f"Unexpected term {t}")

s = z3.Solver()
for (lhs,rhs) in rules:
    s.add(weight(lhs) > weight(rhs))
print(s)
s.check()
#m = s.model()
#m




[f + 0 + __varweight > __varweight,
 plus + 0 + s + 0 + __varweight + __varweight >
 s + 0 + plus + 0 + __varweight + __varweight,
 plus + 0 + 0 + 0 + __varweight > __varweight]


unsat

In [46]:

class LinTerm(dict):
    """ Linear terms like 1 + 2x + 3y implemented as dictionary {None: 1, "x": 2, "y": 3}"""
    def __add__(self, other):
        if isinstance(other, dict): # vector addition
            res = LinTerm(self)
            for k,v in other.items():
                res[k] = res.get(k, z3.IntVal(0)) + v
        else: # scalar addition
            res = LinTerm(self)
            print("here", other)
            res[None] = res.get(None, z3.IntVal(0)) + other
            return res
        return res
    def __mul__(self,other): #scalar multiplication
        if isinstance(other, dict):
            raise ValueError("Multiplication of linear terms not supported")
        res = LinTerm()
        for k,v in self.items():
            res[k] = v * other
        return res
    def __rmul__(self,other):
        return self * other
    def __radd__(self,other):
        return self + other
    def str(self):
        return " + ".join([f"{v} * {k}" for k,v in self.items()])
    def repr(self):
        return " + ".join([f"{v} * {k}" for k,v in self.items()])
    def __lt__(self,other):
        if not isinstance(other, dict):
            raise ValueError("Comparison of linear terms not supported")
        return z3.And([self.get(k,z3.IntVal(0)) < other.get(k,z3.IntVal(0)) for k in set(self.keys()).union(other.keys())])


def poly_interp(head):
    def res(*args):
        arity = str(len(args))
        print("here")
        print(z3.Int(head + "_" + arity + "_" + str(0)))
        for i,a in enumerate(args):
            print(a)
            print( a * z3.Int(head + "_" + arity + "_" + str(i)) + z3.Int(head + "_" + arity))
        return sum(a * z3.Int(head + "_" + arity + "_" + str(i)) for (i,a) in enumerate(args)) + z3.Int(head + "_" + arity) 
    return res

def weight(t):
    if isinstance(t, Var):
        return LinTerm({t.name: 1})
    elif isinstance(t, Fn):
        return poly_interp(t.head)(*map(weight, t.args))
    else:
        raise ValueError(f"Unexpected term {t}")

s = z3.Solver()
for (lhs,rhs) in rules:
    s.add(weight(lhs) > weight(rhs))
s.check()
s.model()

def pp(model):
    interp = {}
    for d in model.decls():
        v = model[d]
        match d.name().split("_"):
            case (head,arity):
                interp[head] = interp.get(head, LinTerm()) + {None: v}
            case (head,arity,i):
                interp[head] = interp.get(head, LinTerm()) + {"x" + str(i): v}

        return interp
pp(s.model())


here
f_1_0
{'x': 1}
here f_1
{'x': 1*f_1_0, None: 0 + f_1}
here 0
here f_1


{'f': {None: 0 + 1}}

In [ ]:
# integer transition systems

# cond -> rhs
x,y,z = z3.Ints("x y z")
vars = (x,y,z)
rules = [
    (x >= 0, (x - 1, y, z)),
    (y >= 0, (x, y - 1, z)),
]

# linear measure ax + by + cz
a,b,c = z3.Ints("a b c")

for constr, (x1,y1,z1) in rules:
    s.add(z3.ForAll([x,y,z], z3.Implies(constr, a*x + b*y + c*z > a*x1 + b*y1 + c*z1)))
s.check()



In [ ]:
import re
lexer = [
    (r"\s+", None),
    (r"\d+", "INT"),
    (r"\w+", "ID"),
    ("RULES", "RULES"),
    ("VAR", "VAR")
    (r"\(", "LPAREN"),
    (r"\)", "RPAREN"),
    (r"->", "ARROW"),
]

re.compiles()

 lexicographic vector measure. f(x) > f(x1) or f(x) == f(x1) and g(x) > g(x1)
 It's all easy enough to state. The quantifier gives me unease though.
 CEGAR it?
 I mean actually a single exaple problem would be nice
 https://aprove.informatik.rwth-aachen.de/interface/v-AProVE2023/itrs
 https://aprove.informatik.rwth-aachen.de/interface/v-AProVE2023/inttrs

 koat and loat are like this right?
 https://arxiv.org/pdf/2202.04546.pdf https://koat.verify.rwth-aachen.de/

 polygonal loops

 f(x,y,z) -> f(x + 1, y - 2)  is a integer transition system.
 changing the top symbol is a way of adding finite state to this.

 https://koat.verify.rwth-aachen.de/
 https://aprove.informatik.rwth-aachen.de/interface/v-koat/c a secret koat web demo oooh
  

Polynomial interpretation does give a weight to each symbol, but also gives a weighting to each possible subterm. That rules.

iteratively apply to prune.

right ground systems have decidable termination.
ground systems also have 

for knuckledragger.
- critical pair checker. No critical pairs = confluent. foo(a,b) = bar(b,c) should be unsat
- termin checker = polynomial interp. structurally decreasing is good, but we can do better. Build in integers
- completeness checker = could just give default value? Isn't unsound persay.



In [69]:
Nat = z3.Datatype("Nat")
Nat.declare("zero")
Nat.declare("s", ("pred", Nat))
Nat = Nat.create()

plus = z3.Function("plus", Nat, Nat, Nat)
x,y = z3.Consts("x y", Nat)
defns = [
    z3.ForAll([x], plus(Nat.zero, x) == x),
    z3.ForAll([x,y], plus(Nat.s(x), y) == Nat.s(plus(x,y)))
]

# more split up and easier to check.
# defining fun sym. pat vars, pattern, rhs
defns2 = plus, [
    ([x], (Nat.zero, x), x),
    ([x,y], (Nat.s(x), y), Nat.s(plus(x,y)))
]

def check_defn(defns):
    for defn in defns:
        assert defn.is_forall()
        for v in defn.num_vars()
            defn.var_name(v)
    defn.body()
    # check symbol undefined 
    # quantifier free

    # check top symbol on lhs is same

    # check no overlap

    # check termination

def check_thm():
    pass



SyntaxError: expected ':' (71271212.py, line 16)

In [74]:
dir(defns[0])
defn = defns[1]
#defn.weight()
dir(defn)
b = defn.body()
#dir(b)


In [ ]:
def subterms(t):
    if isinstance(t, Var):
        return
    yield t
    if isinstance(t, Fn):
        for a in t.args:
            yield from subterms(a)

def capital(t):
    """replace head symbol with new thing. Why?"""
    if isinstance(t, Fn):
        return Fn(t.head.capitalize(), t.args)
    else:
        raise ValueError(f"Unexpected term {t}")
    
def dependency_pairs(lhs,rhs):
    for t in subterms(rhs):
        yield (lhs,t)


In [5]:
%%file /tmp/prolog.pl
%query: append(b,f,f)
append(nil,L,L).
append(cons(X,Xs),Ys,cons(X, Zs)) :- append(Xs,Ys,Zs).

Overwriting /tmp/prolog.pl


In [3]:
! java -ea -cp ~/Documents/solvers/aprove.jar aprove.CommandLineInterface.PrologFrontendMain 

No input file, aborting.
Usage: java -cp aprove.jar aprove.CommandLineInterface.PrologFrontendMain [OPTION] PLFILE

Prolog to QDP/Graph dump export from AProVE 2014.

 -h, --help             print this help
 -o, --outputDir DIR    directory in which TRSs will be dumped (default '.')
 -t, --timeout SECONDS  timeout, in seconds (default 60)
 -p, --proof            print proof for steps from input to TRSs
 -g, --graph yes|no     export to Graph (default no)
 -q, --query QUERY      a query which tells AProVE what to analyze
 -s, --startTerm TERM   analyze termination starting with term TERM
 -j, --json yes|no      export JSON (default no), conflicts with other output


In [6]:
! java -ea -cp ~/Documents/solvers/aprove.jar aprove.CommandLineInterface.PrologFrontendMain /tmp/prolog.pl

Dumped to ./prolog.pl-obl-4.qdp


In [ ]:
(VAR T17 T18)
(PAIRS
  F1_IN(cons(T17, T18)) -> U1^1(f1_in(T18), cons(T17, T18))
  F1_IN(cons(T17, T18)) -> F1_IN(T18)
)
(RULES
  f1_in(nil) -> f1_out1
  f1_in(cons(T17, T18)) -> U1(f1_in(T18), cons(T17, T18))
  U1(f1_out1, cons(T17, T18)) -> f1_out1
)
(Q
)
(EDGES
    2 -> 1
    2 -> 2
)
(MINIMAL)

In [7]:
%%file /tmp/hask.hs
{-# htermination (foldr1 :: (a -> a -> a) -> (List a) -> a) #-}


import qualified Prelude


data MyBool = MyTrue | MyFalse

data List a = Cons a (List a) | Nil


foldr1 :: (a -> a -> a) -> (List a) -> a

foldr1 f (Cons x Nil) = x

foldr1 f (Cons x xs) = f x (foldr1 f xs)

Overwriting /tmp/hask.hs


In [9]:
! java -ea -cp ~/Documents/solvers/aprove.jar aprove.CommandLineInterface.HaskellFrontendMain  /tmp/hask.hs

Dumped to ./hask.hs-obl-14.qdp
Dumped to ./hask.hs-obl-16.qdp


In [ ]:
(VAR vx3 vx40 vx410 vx411 h)
(PAIRS
  new_foldr1(vx3, Cons(vx40, Cons(vx410, vx411)), h) -> new_foldr1(vx3, Cons(vx410, vx411), h)
)
(RULES
)
(Q
)
(EDGES
    1 -> 1
)
(MINIMAL)

Termination is the question of whether some dynamical system will always eventually stop.
The "dyanamical system" in question is usually a term rewriting system (rules describing simplifying or evaluating exressions like `x + 0 -> x` or `append(cons(a,x), y) -> cons(append(x,y))`). The state here is the current term and the dynamics are described by the rules. There are an infinite number of posssible terms/states to consider here. What is there to do?

An old notion of termination is that of a fixed point. The differential equation `y' = -y` has solutions $y = C e ^ {-x}$. We can see that this is a decaying solution heading to `y = 0`. We could also find this fixed point by finding where `'y = 0` which mean `y = 0`.
Lyapunov functions are certificates for regions of decay.

It perhaps helps to consider a system with a finite number of states. It then is clearly more tractable to consider the question of whether a system will eventually get stuck. If we think about it, this is equivalent to just checking if there is a loop in the transition graph.

For infinite state systems the problem is more tricky. We can't exhaustively be sure there are no loops by just checking each state.

Termination is important in mathematics and program analysis. If you alow a non-terminating definition of a function into your logical system
For program analysis, if you can guarantee some loop doesn't terminate, you can ignore whatever comes after it. If you can guarantee it always terminates, you can perhaps lift code that comes after to before the loop or rearrange call sites. A guaranteed terminating piece of pure code behaves more like an ideal mathematical function and is more easy to reason and manipulate correctly. If a compiler can't reason, it is not allowed to perform optimizatins and may emit pessimistic code. This may produce better code from a compiler. Having some exact bounds of the maximum number of iterations may unlock even more. For some loops (like fixed parameter for loops) this may be very easy to do, for some nasty while loop it may not.
For verification purposes, it may be desirable to state and prove that a function always returns an answer with such and such result.

There is a class of solver called a termination checker

The core notion of a measure is some natural number that goes down and is bounded below by zero, like the length of a list. With enough squinting, other termination certificates are generalizations of this intuitive idea.

The idea of a well-founded relation is in essence trying to just define what it means for something to eventually always stop and our basic notion of something that always stops is a decreasing natural number.

<https://cstheory.stackexchange.com/questions/34316/is-there-a-good-notion-of-non-termination-and-halting-proofs-in-type-theory>

`zero = cons(0,zero)` is "ok". It has a model N -> N? Maybe non terminating definitions can interfere with the foundation axiom in some bad way. or `fact(n) = n == 0? 1 : n * fact(n-1)` when `n < 0` runs off to infinity, but maybe some ordinal or hyperinteger model can handle it?